# Simular el Modelo  de HW en la Medida Forward

## Encontrar el Nuevo *Drift*

Primero se debe encontrar el nuevo drift $m^*\left(r_t,t\right)+\sigma_Z\left(r_t,t\right)\sigma\left(r_t,t\right)$ con

$$
\begin{equation}
\sigma_{Z}\left(r_t,t\right)=\frac{\partial Z}{\partial r}\frac{1}{Z}\sigma\left(r_t,t\right)
\end{equation}
$$

Recordar que en el modelo de HW:

$$
m^*\left(r_t,t\right)=\theta_t-\gamma^* r_t
$$

Y la fórmula para el valor de un bono cupón cero es:

$$
Z\left(r_t,t,T\right)=\exp\left[A\left(t,T\right)-B\left(t,T\right)r_t\right]
$$

Por lo tanto,

$$
\frac{\partial Z}{\partial r}=-B\left(t,T\right)Z
$$

Lo que implica que,

$$
\sigma_{Z}\left(r_t,t\right)=-B\left(t,T\right)\sigma
$$

Finalmente, el drift en la medida forward es:

$$
\theta_t-B\left(t,T\right)\sigma^2-\gamma^* r_t
$$

Para simular, puedo utilizar las funciones del notebook 13, considerando un nuevo $\theta_t$ sea éste $\overline{\theta_t}$, dado por la siguiente fórmula:

$$
\overline{\theta_t}=\theta_t-B\left(t,T\right)\sigma^2
$$

## Simular

In [1]:
from scipy.interpolate import interp1d
import plotly.express as px
import pandas as pd
import numpy as np
import math

Se obtienen los valores de la curva cero cupón:

In [2]:
curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [3]:
curva['t'] = curva['plazo'] / 365.0

In [4]:
curva['log_df'] = np.log(curva['df'])

In [5]:
curva.head()

,plazo,tasa,df,t,log_df
0,1,0.000811,0.999998,0.002740,-0.000002
1,7,0.000841,0.999984,0.019178,-0.000016
2,14,0.000780,0.999970,0.038356,-0.000030
3,21,0.000774,0.999955,0.057534,-0.000045
4,33,0.000781,0.999929,0.090411,-0.000071


La curva se obtiene interpolando linealmente en los factores de descuento. Esto es una novedad respecto al notebook 13.

In [6]:
dfcurva = interp1d(curva['t'],
                  curva['tasa'],
                  kind='cubic',
                  fill_value="extrapolate")
def zrate(t: float):
    return dfcurva(t)

In [7]:
zrate(2)

array(0.00056758)

Se definen las derivadas de la curva.

In [8]:
def dzrate(t: float) -> float:
    delta = .0001
    return (zrate(t + delta) - zrate(t)) / delta


def d2zrate(t: float) -> float:
    delta = .0001
    return (dzrate(t + delta) - dzrate(t)) / delta

In [9]:
t = 2
print(f'zrate({t:.2f}) = {zrate(t):.4%}')
print(f'dzrate({t:.2f}) = {dzrate(t):.4%}')
print(f'd2zrate({t:.2f}) = {d2zrate(t):.4%}')

zrate(2.00) = 0.0568%
dzrate(2.00) = -0.0066%
d2zrate(2.00) = 0.0584%


In [10]:
def fwd(t: float) -> float:
    return zrate(t) + t * dzrate(t)

def dfwd(t: float) -> float:
    return 2 * dzrate(t) + t * d2zrate(t)

In [11]:
print(f'fwd({t:.2f}) = {fwd(t):.4%}')
print(f'dfwd({t:.2f}) = {dfwd(t):.4%}')

fwd(2.00) = 0.0436%
dfwd(2.00) = 0.1036%


In [12]:
sigma = .015
gamma = .5

# Proxy de la tasa instantánea r(t) es una tasa entre t y t + dt (donde dt es un intervalo infitesimal)
r0 = zrate(1/264)
print(r0)

0.0008182064199498154


In [13]:
def b_hw(gamma: float, t: float, T: float) -> float:
    """
    Calcula el valor de la función B(t,T) que interviene en la fórmula
    para el valor de un bono cupón cero en el modelo de HW.
    
    params:
    
    - gamma: intensidad de reversión del modelo HW
    - t:
    - T:
    
    return:
    
    - valor de la función B(t, T)
    """
    aux = 1 - math.exp(- gamma * (T - t))
    return aux / gamma

In [14]:
def theta_fwd_measure(t: float, T:float) -> float:
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux - b_hw(gamma, t, T)*sigma**2

In [15]:
def sim_hw_many(gamma, sigma, theta_fwd_measure, T, r0, num_sim, num_steps, seed = None):
    """
    """
    dt = 1 / 264.0
    num_steps += 1
    
    # Calcula los números aleatorios
    alea = np.zeros((num_sim, num_steps))
    np.random.seed(seed)

    for i in range(0, num_sim):
        for j in range(0, num_steps):
            alea[i][j] = np.random.normal()
            
    # Calcula los valores de Theta. Theta sólo depende del tiempo, no de la simulación. 
    theta_array = np.zeros(num_steps)
    tiempo = np.zeros(num_steps)
    for i in range(1, num_steps):
        tiempo[i] = i * dt
        theta_array[i] = theta_fwd_measure(i * dt, T)
    
    # Simula las trayectorias
    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma * dt
    sim = np.zeros((num_sim, num_steps))
    for i in range(0, num_sim):
        sim[i][0] = r0
        r = r0
        for j in range(1, num_steps):
            r = r + theta_array[j - 1] * dt - gamma_dt * r + sqdt_sigma * alea[i][j - 1]
            sim[i][j] = r
    return tiempo, sim

In [16]:
num_sim = 1000
num_steps = 264
seed = 1234
df = 0
T = 1
tiempo, s = sim_hw_many(gamma, sigma, theta_fwd_measure, T, r0, num_sim, num_steps, seed)

In [27]:
which_sim = 999
result = [(z[0], z[1]) for z in zip(tiempo, s[which_sim][0:])]
df_sim = pd.DataFrame(result, columns=['plazo', 'tasa'])

In [28]:
fig = px.line(
    df_sim,
    x='plazo',
    y='tasa',
    title=f'Sim HW en Medida Fwd'
)
fig.show()

## Valorizar una Ballena

La *ballena* es una especie de call sobre un bono cupón cero. Supongamos que:

- bono cero cupón subyacente es el que vence hoy en 2Y más.
- Valorizar para strikes: 98, 99, 100, 101, 102.
- Usar 1000 simulaciones.
- Los precios del bono cupón cero hay que multiplicarlos por 100 antes de calcular el payoff.

Para cada simulación, con la última tasa de la simulación, calcular $z$ e insertarlo en el payoff de la ballena. Calcular el promedio de esos resultados. Traer el promedio a valor presente con el factor de descuento a 1Y de la curva.

In [19]:
def ballena(z: float, strike: float) -> float:
    """
    z: es el precio del bono cupón cero al vencimiento
    strike: es el strike de la opción
    """
    return max(math.log(max(z - strike, .0001)), .0001)

In [20]:
result = [(i, ballena(i, 95)) for i in range(90, 111)] # list comprehensions
df_ballena = pd.DataFrame(result, columns=['precio', 'valor'])

In [21]:
fig = px.line(
    df_ballena,
    x='precio',
    y='valor',
    title=f'Payoff de una Ballena sobre un Bono Cero Cupón'
)
fig.show()

### Extrae la Última Tasa de Cada Simulación

In [41]:
last_rates = [sim[-1] for sim in s]

In [42]:
which_sim = 50
print(f'Última tasa de la simulación {which_sim}: {last_rates[which_sim]: .4%}')

Última tasa de la simulación 50: -0.4078%


### Calcula el Precio del Bono Cupón Cero

Traemos la función $A$:

In [43]:
def a_hw(zrate: float, fwd, gamma: float, sigma: float, t: float, T: float,
         verbose = False):
    """
    verbose: cuando es True imprime los valores de c1, c2 y c3.
    """
    b = b_hw(gamma, t, T)
    dfT = math.exp(-zrate(T) * T)
    dft = math.exp(-zrate(t) * t)
    c1 = math.log(dfT / dft)
    c2 = b * fwd(t)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t))
    if verbose:
        print("c1: " + str(c1))
        print("c2: " + str(c2))
        print("c3: " + str(c3))
    return c1 + c2 - c3

Y la función para el precio del bono cupón cero:

In [80]:
def zero_hw(r: float,
            gamma: float,
            sigma: float,
            zrate: float,
            fwd: float,
            t: float,
            T: float) -> float:
    a = a_hw(zrate, fwd, gamma, sigma, t, T)
    b = b_hw(gamma, t, T)
    return math.exp(a - b * r)

In [81]:
precios = [zero_hw(r, gamma, sigma, zrate, fwd, t=1, T=2) * 100 for r in last_rates]

In [82]:
which_sim = 1
print(f'Valor bono cupón cero con la simulación {which_sim}: {precios[which_sim]: .4f}')

Valor bono cupón cero con la simulación 1:  99.9852


### Calcula Payoffs

In [85]:
strikes = [98, 99, 100, 101, 102]

In [86]:
payoffs = {strike:[ballena(z, strike) for z in precios] for strike in strikes}

### Calcula Valores Presente

In [87]:
valores = [math.exp(-1.0 * float(zrate(1))) * np.average(payoffs[strike]) for strike in strikes]

In [88]:
for v in valores:
    print(f'valor: {v:.8f}')

valor: 0.63402728
valor: 0.23224051
valor: 0.04233673
valor: 0.00351143
valor: 0.00009993
